## Домашнее задание по блоку 01.04.2021. Случайные события и случайные величины
### Тихонова Инна

In [1]:
import pandas as pd
import numpy as np

Смоделировать игру против лотерейного автомата типа “777”. Игрок платит 1 руб., после чего выпадает случайное целое число, равномерно распределенное от 0 до 999. При некоторых значениях числа игрок получает выигрыш:
 - 777: 200 руб.
 - 999: 100 руб.
 - 555: 50 руб.
 - 333: 15 руб.
 - 111: 10 руб.
 - *77: 5 руб.
 - **7: 3 руб.
 - *00: 2 руб.
 - **0: 1 руб.

1. Выгодна ли игра игроку?
2. Сколько в среднем приобретает или теряет игрок за одну игру?

Рассчитаем вероятность выигрыша и средний выигрыш игрока по методу Монте-Карло:

In [2]:
def monte_carlo_n(trials, payment=0):

    monte_carlo = []
    successes = 0
    successes_more_1 = 0

    for i in range(trials):

        random_score = np.random.randint(0, 1000)
        outcome = 0

        if random_score == 777:
            outcome = 200
        elif random_score == 999:
            outcome = 100
        elif random_score == 555: 
            outcome = 50
        elif random_score == 333:
            outcome = 15
        elif random_score == 111:  
            outcome = 10

        elif random_score % 100 == 77:  
            outcome = 5
        elif random_score % 10 == 7:  
            outcome = 3 
   
        elif random_score % 100 == 0:  
            outcome = 2 
        elif random_score % 10 == 0:     
            outcome = 1 

        if outcome != 0:
            successes += 1
        if outcome > 1:
            successes_more_1 += 1    
            
        monte_carlo.append(outcome + payment)    


    print("Вероятность выигрыша ненулевой суммы за 1 ход: ", successes / trials)
    print("Вероятность выигрыша суммы более 1 рубля за 1 ход: ", successes_more_1 / trials)
    
    monte_carlo = pd.DataFrame(columns=['data'], data=monte_carlo)
    return monte_carlo
    

Проведем 1000 экспериментов - по количеству возможных исходов игры:

In [3]:
monte_carlo = monte_carlo_n(1000, -1)

Вероятность выигрыша ненулевой суммы за 1 ход:  0.216
Вероятность выигрыша суммы более 1 рубля за 1 ход:  0.119


Получаем, что вероятность выиграть хоть что-то - примерно 1 / 5. При этом вероятность выиграть сумму более 1 рубля - чуть больше 1 / 10.

Посчитаем средний и суммарный выигрыш игрока за эти 1000 ходов:

In [4]:
print(np.mean(monte_carlo))

data   -0.248
dtype: float64


In [5]:
monte_carlo.sum()

data   -248
dtype: int64

В среднем игрок теряет за игру несколько копеек (с учетом уплаты 1 рубля). А суммарно теряет даже те небольшие деньги, которые удалось выиграть и уходит в минус.

Посмотрим общие характеристики полученного массива:

In [6]:
monte_carlo.describe().T

,count,mean,std,min,25%,50%,75%,max
data,1000.0,-0.248,5.540866,-1.0,-1.0,-1.0,-1.0,99.0


Видим, что максимальное значение = 99 рублей (100 рублей минус 1 рубль за игру), т.е. нам удалось выиграть почти максимальную сумму. При этом все остальные показатели говорят о неравномерности распределения: 25, 50 и 75% квартили находятся на -1 (0 рублей минус 1 рубль за игру).

Проверим все то же самое, но на большем количестве попыток:

In [7]:
monte_carlo = monte_carlo_n(1000000, -1)

Вероятность выигрыша ненулевой суммы за 1 ход:  0.203947
Вероятность выигрыша суммы более 1 рубля за 1 ход:  0.114168


Снова получаем вероятность выигрыть хоть что-то равной 1 / 5, а вероятность выиграть сумму более 1 рубля около 1 / 10.

Посчитаем средний и суммарный выигрыш игрока за эти 1 000 000 ходов:

In [8]:
print(np.mean(monte_carlo))

data   -0.207189
dtype: float64


In [9]:
monte_carlo.sum()

data   -207189
dtype: int64

Снова в минусе, хотя и суммы немного другие. Смотрим общие характеристики:

In [10]:
monte_carlo.describe().T

,count,mean,std,min,25%,50%,75%,max
data,1000000.0,-0.207189,7.196885,-1.0,-1.0,-1.0,-1.0,199.0


Ничего не поменялось: ни среднее, ни квартили. Почти всегда выигрыш игрока будет равен -1 (0 рублей минус 1 рубль за игру).

Посчитаем теоретическую вероятность выигрыша по формуле полной вероятности:

In [11]:
#событие A - выиграли любую ненулевую сумму
p_a = 0
#событие A_more_1 - выиграли сумму более 1 рубля
p_a_more_1 = 0

#вероятности событий
p_h = []

#события = [выпало 777 ; выпало 999 ; выпало 555 ; выпало 333 ; выпало 111]
#вероятность одного такого события = 1/1000, всего таких событий 5
for _ in range(5):
    p_h.append(1 / 1000)

#события = [выпало 077 , выпало 177 , ...,  выпало 977;
#          [выпало 000 , выпало 100 , ...,  выпало 900]
#вероятность одного такого события = 1/100, всего таких событий 2    
for _ in range(2):
    p_h.append(1 / 100)


#события = [выпало 007 , выпало 017 , ...,  выпало 997;
#          [выпало 010 , выпало 020 , ...,  выпало 990]
#вероятность одного такого события = (1 / 10 - 1/100), всего таких событий 2    
for _ in range(2):
    p_h.append(1 / 10 - 1 / 100)    

#вероятность выигрыша при каждом из событий в массиве p_h = 1

#все остальные возможные исходы (123, 456 и т.д.) имеют вероятность 1 / 1000, а вероятность выигрыша при таких исходах = 0,
#поэтому мы эти исходы не включаем в формулу

p_a = sum(p_h)
print("Теоретическая вероятность выигрыша ненулевой суммы: ", p_a)

p_a_more_1 = p_a - (1 / 10 - 1 / 100)
print("Теоретическая вероятность выигрыша суммы более 1 рубля: ", p_a_more_1)
    

Теоретическая вероятность выигрыша ненулевой суммы:  0.20500000000000002
Теоретическая вероятность выигрыша суммы более 1 рубля:  0.115


Получили схожие результаты: вероятность выиграть хоть что-то в районе 1 / 5, вероятность выиграть сумму более 1 рубля - около 1 / 10.